# Exercise 1 on Clustering: Synthetic Dataset

The goal of the exercise is to perform perform clustering on a dataset representing a noisy 2D non-linear function.
* How do we select the correct number of clusters?
* Why LPCA works better than PCA on a non-linear dataset?

First thing, we create our synthetic dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits

def non_linear_func_orig(x):
    y = np.sin(x) + np.sin(1.5*x)
    return y

def non_linear_func(x):
    y = np.sin(x) + np.sin(1.5*x) + 0.25*np.random.randn(x.size) 
    return y
    
# This code is used to create our synthetic dataset
size = 1000
seed = 42
np.random.seed(seed)

x = np.random.rand(size) * 2.6*np.pi - np.pi/2
y = non_linear_func(x)

limits = [np.pi/2.5, 1.2*np.pi]
eps = np.pi/10

mask = ((x < limits[0] + eps) & (x > limits[0] - eps)) | (x < limits[1] + eps) & (x > limits[1] - eps)
x = x[~mask]
y = y[~mask]

X = np.zeros((x.size, 2))
X[:,0] = x
X[:,1] = y

x_orig = np.linspace(x.min(), x.max(), 200)
y_orig = non_linear_func_orig(x_orig)

# We can plot the dataset and the original non-linear function
plt.scatter(X[:,0], X[:,1], c='b', alpha=0.5)
plt.plot(x_orig, y_orig, c='cyan', lw=5)
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Global PCA
Now we apply the PCA and try to reconstruct the dataset using only one dimension.
 - To do:
    - obtain X_std (dataset preprocessed with mean and STD)
    - apply PCA
    - recontruct the solution with one dimension (q=1)
        -hint: 
            - X     = Z  @ A.T
            - X_rec = Zq @ Aq.T 
    - plot the reconstruction vs the original dataset


 - hint: the documentation and examples to use the PCA object are at:
       https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
# We import the PCA object from the sklearn package
from sklearn.decomposition import PCA

# scale -> X_std
# ...

# Apply PCA
# ...

# Reconstruct with 1 PC
# ...

# plot the reconstruction VS original
# ...

## Clustering

We can try and improve the reconstruction accuracy using a local approach. The first thing to do is then to cluster the dataset using kmeans.
- To do: 
    - Use KMeans object to obtain the cluster labels or indexes (=`idx`). init = 'random' and n_init='auto'
    - Use a number of clusters (k) of your choice.
    - Plot the dataset and colour it by the value of the label
   
   
- Hint: the documentation and examples to use the PCA object are at:
       https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
# We import the KMEANS object from the sklearn package
from sklearn.cluster import KMeans

# Number of clusters
# ...

# Perform KMeans
# ...

# Plot the clustering
# # Uncomment the lines below when you have calculated the clusters' labels (=idx).
# plt.scatter(X[:,0], X[:,1], c=labels)
# plt.show()

## Selection of the clusters' number

In this case we can visually identify the number of clusters, because we have only two dimensions. In higher dimensions, it is not possible to visualize the dataset. Also, we would prefer to have an automatic method to select the correct number of clusters.

We can use the Davies Bouldin score to estimate the number of clusters.

- To do: 
    - Calculate the DB score for a range of k (number of clusters) that goes from 2 to 10.
    - Plot the DB score as a function of k.
    - Select n_clusters based on the DB score
   
   
- Hint: 
    - the documentation and examples to use the PCA object are at:
    https://scikit-learn.org/stable/modules/generated/sklearn.metrics.davies_bouldin_score.html
    - you can use a 'for' loop to calculate the DB for different k

In [ ]:
from sklearn.metrics import davies_bouldin_score

# Create the k_array and the db_score_array, initiliazed with zeros
# ...

# Loop on k where: 1) perform KMeans, 2) calculate db_score and 3) stores them in db_score_array
# for i,k in enumerate(k_array):
# ...


# Plot DB score Vs n_clusters
# ...

# Select n_clusters based on db_scores
# ...

print(f'n_clusters = {n_clusters}')

## Local PCA

Now we can apply the PCA in each cluster.
- To do: 
    - apply PCA for each cluster
    - plot the reconstruction using only the first PC.
- Hint:
    - Use a 'for' loop to apply the PCA in each cluster.
    - to select only the data of a single cluster use a mask:
           To select the data with label 0:
             mask = (idx == 0)
             X_k = X[mask]
    - To get the correct reconstruction, you need to **center and scale** the data **in each cluster** before applying the PCA, and then de-center and de-scale the data

In [ ]:
# We compute Kmeans with the correct number of clusters
# ...         

q = 1

# We create the PCA object
# ...         

# Loop on clusters:
# for k in range(n_clusters):
#    print(f' k = {k}')
    
    # create the mask: select the points belonging to cluster k
    # ...         

    
    # center and scale X_k
    # ...         

    
    # perform the pca
    # ...         
    
    # recontruct and uncenter and unscale
    # ...         

    
    # plot the reconstruction
    #plt.scatter(X_k[:,0]    ,X_k[:,1],     c='b', alpha=0.5)
    #plt.scatter(X_k_rec[:,0],X_k_rec[:,1], c='r', alpha=0.5)

#plt.plot(x_orig, y_orig, c='cyan', lw=5, alpha=0.7)
#plt.show()

## Local PCA with VQPCA

We can compare the KMEANS results with VQPCA. To perform VQPCA, we need the [OpenMORe](https://github.com/burn-research/OpenMORe) library. You can run the next cell to install it. 

In [ ]:
!git clone https://github.com/burn-research/OpenMORe.git

import sys

sys.path.insert(0,'/content/OpenMORe')

import OpenMORe

In [ ]:
import OpenMORe.clustering as clustering

# This is a dictionary used to set the VQPCA parameters.
settings_clustering = {
    #centering and scaling options
    "center"                    : True,
    "centering_method"          : "mean",
    "scale"                     : True,
    "scaling_method"            : "auto",

    #set the initialization method (random, observations, kmeans, pkcia, uniform)
    "initialization_method"     : "random",

    #set the number of clusters and PCs in each cluster
    "number_of_clusters"        : 3,
    "number_of_eigenvectors"    : 1,

    #enable additional options:
    "correction_factor"         : "off",    # --> enable eventual corrective coefficients for the LPCA algorithm:
                                            #     'off', 'c_range', 'uncorrelation', 'local_variance', 'phc_multi', 'local_skewness' are available

    "classify"                  : False,    # --> call the method to classify a new matrix Y on the basis of the lpca clustering
    "write_on_txt"              : False,    # --> write the idx vector containing the label for each observation
    "evaluate_clustering"       : True,     # --> enable the calculation of indeces to evaluate the goodness of the clustering
    "neighbors_number"          : 0,
    
    "kNN_post"                  : False,     # activate the kNN algorithm once the convergence is achieved
}

# To do: follow the example in 
# https://github.com/burn-research/OpenMORe/blob/master/examples/clustering/lpcaExample.py

# to find the vqpca labels, then plot the clusters

# ...